First, we start of by importing the necessary python libraries.

In [72]:
import pandas as pd

Next step is importing our data files and running transformation operations, we will start by removing the unnecessary columns

In [73]:
movies_revenues=pd.read_csv("movies_metadata.csv")
engagement=pd.read_excel("netflix_engagement_report.xlsx")
netflixlib=pd.read_csv("netflix_library.csv")
ratings= pd.read_table("imdb_ratings.tsv")
netflixlibcredits=pd.read_csv("netflix_library_credits.csv")

movies_revenues.drop(['adult','runtime','original_language','production_companies','belongs_to_collection','genres','homepage','id', 'runtime',
              'original_title','overview','popularity',
              'poster_path','production_countries',
              'spoken_languages','status','tagline','video','title','vote_average','vote_count','release_date'],
              axis=1, inplace=True)
netflixlib.drop(['description','imdb_score','imdb_votes','tmdb_score','seasons','tmdb_popularity','runtime'], axis=1, inplace=True )
netflixlibcredits.drop(['character'], axis=1, inplace=True )
ratings.drop(['numVotes'], axis=1, inplace=True )


C:\Users\aziba\AppData\Local\Temp\ipykernel_18696\1980833224.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_revenues=pd.read_csv("movies_metadata.csv")


movies_revenues: this table holds revenue and budget information for a number of movies

In [74]:
#remove rows with values of 0 in budget and revenue
movies_revenues = movies_revenues[(movies_revenues['budget'] != '0') & (movies_revenues['revenue'] != 0) ]
#remove rows with empty values
movies_revenues = movies_revenues.dropna(subset=['budget', 'revenue'], how='any')
#changing the datatype of the budget column
movies_revenues['budget'] = movies_revenues['budget'].astype(float)
#adding the profit column
movies_revenues['profit'] = movies_revenues['revenue'] - movies_revenues['budget']
#inserting a new index
movies_revenues.insert( 0, 'revenue_id', movies_revenues.imdb_id)
movies_revenues.drop('imdb_id', axis=1, inplace=True )
#remove any duplicates
movies_revenues= movies_revenues.drop_duplicates()
display(movies_revenues)
movies_revenues.info() 

,revenue_id,budget,revenue,profit
0,tt0114709,30000000.0,373554033.0,343554033.0
1,tt0113497,65000000.0,262797249.0,197797249.0
3,tt0114885,16000000.0,81452156.0,65452156.0
5,tt0113277,60000000.0,187436818.0,127436818.0
8,tt0114576,35000000.0,64350171.0,29350171.0
...,...,...,...,...
45250,tt0479751,12000000.0,19000000.0,7000000.0
45399,tt3805180,750000.0,3.0,-749997.0
45409,tt0933361,800000.0,1328612.0,528612.0
45412,tt1718881,2000000.0,1268793.0,-731207.0


<class 'pandas.core.frame.DataFrame'>
Index: 5375 entries, 0 to 45422
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   revenue_id  5375 non-null   object 
 1   budget      5375 non-null   float64
 2   revenue     5375 non-null   float64
 3   profit      5375 non-null   float64
dtypes: float64(3), object(1)
memory usage: 210.0+ KB


4. netflixlib (library): this table holds records for shows and movies available on Netflix

In [75]:

#removing rows with empty ID and title values
netflixlib = netflixlib.dropna(subset=['imdb_id'], how='any')
netflixlib = netflixlib.dropna(subset=['title'], how='any')
#removing duplicates where there are two movies or shows with the same title released on the same year
netflixlib = netflixlib.drop_duplicates(subset=['title', 'release_year'], keep='first')
netflixlib.insert(0, 'movie_id', netflixlib.imdb_id)
display(netflixlib)
#splitting the array-like data into regular string values
netflixlib.loc[:, 'production_countries'] = netflixlib['production_countries'].str[2:-2].str.split(', ').str.join(', ')
netflixlib.loc[:, 'production_countries'] = netflixlib['production_countries'].str.replace("'", '') 
netflixlib.loc[:, 'genres'] = netflixlib['genres'].str[2:-2].str.split(', ').str.join(', ')
netflixlib.loc[:, 'genres'] = netflixlib['genres'].str.replace("'", '') 
netflixlib.drop('imdb_id', axis=1, inplace=True )
display(netflixlib)
netflixlib.info()

,movie_id,tmdb_id,title,type,release_year,age_certification,genres,production_countries,imdb_id
1,tt0075314,tm84618,Taxi Driver,MOVIE,1976,R,"['drama', 'crime']",['US'],tt0075314
2,tt0068473,tm154986,Deliverance,MOVIE,1972,R,"['drama', 'action', 'thriller', 'european']",['US'],tt0068473
3,tt0071853,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,"['fantasy', 'action', 'comedy']",['GB'],tt0071853
4,tt0061578,tm120801,The Dirty Dozen,MOVIE,1967,NaN,"['war', 'action']","['GB', 'US']",tt0061578
5,tt0063929,ts22164,Monty Python's Flying Circus,SHOW,1969,TV-14,"['comedy', 'european']",['GB'],tt0063929
...,...,...,...,...,...,...,...,...,...
5843,tt14216488,tm1097142,My Bride,MOVIE,2021,NaN,"['romance', 'comedy', 'drama']",['EG'],tt14216488
5845,tt13857480,tm1014599,Fine Wine,MOVIE,2021,NaN,"['romance', 'drama']",['NG'],tt13857480
5846,tt11803618,tm898842,C/O Kaadhal,MOVIE,2021,NaN,['drama'],[],tt11803618
5847,tt14585902,tm1059008,Lokillo,MOVIE,2021,NaN,['comedy'],['CO'],tt14585902


,movie_id,tmdb_id,title,type,release_year,age_certification,genres,production_countries
1,tt0075314,tm84618,Taxi Driver,MOVIE,1976,R,"drama, crime",US
2,tt0068473,tm154986,Deliverance,MOVIE,1972,R,"drama, action, thriller, european",US
3,tt0071853,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,"fantasy, action, comedy",GB
4,tt0061578,tm120801,The Dirty Dozen,MOVIE,1967,NaN,"war, action","GB, US"
5,tt0063929,ts22164,Monty Python's Flying Circus,SHOW,1969,TV-14,"comedy, european",GB
...,...,...,...,...,...,...,...,...
5843,tt14216488,tm1097142,My Bride,MOVIE,2021,NaN,"romance, comedy, drama",EG
5845,tt13857480,tm1014599,Fine Wine,MOVIE,2021,NaN,"romance, drama",NG
5846,tt11803618,tm898842,C/O Kaadhal,MOVIE,2021,NaN,drama,
5847,tt14585902,tm1059008,Lokillo,MOVIE,2021,NaN,comedy,CO


<class 'pandas.core.frame.DataFrame'>
Index: 5444 entries, 1 to 5849
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   movie_id              5444 non-null   object
 1   tmdb_id               5444 non-null   object
 2   title                 5444 non-null   object
 3   type                  5444 non-null   object
 4   release_year          5444 non-null   int64 
 5   age_certification     3068 non-null   object
 6   genres                5444 non-null   object
 7   production_countries  5444 non-null   object
dtypes: int64(1), object(7)
memory usage: 382.8+ KB


This section is dedicated to forming the table engagement: A merged table of the netflix library and the netflix engagement report table.
<br>-Starting by standardizing the titles so they could be merged with the other dataset.
<br>-We also performed an aggregation operation to sum the amount different seasons of the same tv show were viewed as the original data had each tv show season with its own hours_viewed number. 

In [76]:
transformed_titles = []
#remove rows with empty dates
engagement = engagement.dropna(subset=['release_date'], how='any')

for title in engagement['title']:
    # Remove ": Season" and everything after it
    if ': Season' in title:
        title = title.split(': Season')[0].strip()
    # Remove "//" and everything after it
    if '//' in title:
        title = title.split('//')[0].strip() 
    # Add the transformed title to the list
    transformed_titles.append(title)
#remove rows with empty titles and dates
engagement = engagement[(engagement['title'] != '0')]

# replace the titles with their new values
engagement['title'] = transformed_titles
#change the release_date to release_year only
engagement.insert(1,'release_year' ,pd.to_datetime(engagement["release_date"]).dt.year)

# calculate the sum of hours_viewed for each show across their different seasons
engagement = engagement.groupby('title').agg(
    hours_viewed=('hours_viewed', 'sum'),
    global_availability=('global_availability', 'first'),
    release_year=('release_year', 'first')
).reset_index()

#merging the engagement data with the our netflix library
engagement = engagement.merge(netflixlib, on=['title', 'release_year'], how='right')
engagement = engagement.drop_duplicates()
display(engagement)

,title,hours_viewed,global_availability,release_year,movie_id,tmdb_id,type,age_certification,genres,production_countries
0,Taxi Driver,NaN,NaN,1976,tt0075314,tm84618,MOVIE,R,"drama, crime",US
1,Deliverance,NaN,NaN,1972,tt0068473,tm154986,MOVIE,R,"drama, action, thriller, european",US
2,Monty Python and the Holy Grail,NaN,NaN,1975,tt0071853,tm127384,MOVIE,PG,"fantasy, action, comedy",GB
3,The Dirty Dozen,NaN,NaN,1967,tt0061578,tm120801,MOVIE,NaN,"war, action","GB, US"
4,Monty Python's Flying Circus,NaN,NaN,1969,tt0063929,ts22164,SHOW,TV-14,"comedy, european",GB
...,...,...,...,...,...,...,...,...,...,...
5439,My Bride,NaN,NaN,2021,tt14216488,tm1097142,MOVIE,NaN,"romance, comedy, drama",EG
5440,Fine Wine,NaN,NaN,2021,tt13857480,tm1014599,MOVIE,NaN,"romance, drama",NG
5441,C/O Kaadhal,NaN,NaN,2021,tt11803618,tm898842,MOVIE,NaN,drama,
5442,Lokillo,NaN,NaN,2021,tt14585902,tm1059008,MOVIE,NaN,comedy,CO


Creating new dimension tables off of the engagment table <br> genres: table holding the genres for each tv show/movie <br> regions: table holding the production countries for each tv show/movie

In [77]:
#forming new dimension tables
genres = engagement[['movie_id', 'genres']]
regions = engagement[['movie_id', 'production_countries']]
#renaming ID columns
genres = genres.rename(columns={'movie_id': 'genre_id'})
regions = regions.rename(columns={'movie_id': 'region_id', 'production_countries': 'regions'})
display(genres)
display(regions)
#removing genres and type from engagement table as they are no longer needed
engagement.drop(['genres','production_countries'], axis=1, inplace=True)
#re-ordering columns
engagement = engagement.reindex(columns=['movie_id','tmdb_id','title','hours_viewed','global_availability','release_year','type','age_certification'])
display(engagement)
engagement.info()

,genre_id,genres
0,tt0075314,"drama, crime"
1,tt0068473,"drama, action, thriller, european"
2,tt0071853,"fantasy, action, comedy"
3,tt0061578,"war, action"
4,tt0063929,"comedy, european"
...,...,...
5439,tt14216488,"romance, comedy, drama"
5440,tt13857480,"romance, drama"
5441,tt11803618,drama
5442,tt14585902,comedy


,region_id,regions
0,tt0075314,US
1,tt0068473,US
2,tt0071853,GB
3,tt0061578,"GB, US"
4,tt0063929,GB
...,...,...
5439,tt14216488,EG
5440,tt13857480,NG
5441,tt11803618,
5442,tt14585902,CO


,movie_id,tmdb_id,title,hours_viewed,global_availability,release_year,type,age_certification
0,tt0075314,tm84618,Taxi Driver,NaN,NaN,1976,MOVIE,R
1,tt0068473,tm154986,Deliverance,NaN,NaN,1972,MOVIE,R
2,tt0071853,tm127384,Monty Python and the Holy Grail,NaN,NaN,1975,MOVIE,PG
3,tt0061578,tm120801,The Dirty Dozen,NaN,NaN,1967,MOVIE,NaN
4,tt0063929,ts22164,Monty Python's Flying Circus,NaN,NaN,1969,SHOW,TV-14
...,...,...,...,...,...,...,...,...
5439,tt14216488,tm1097142,My Bride,NaN,NaN,2021,MOVIE,NaN
5440,tt13857480,tm1014599,Fine Wine,NaN,NaN,2021,MOVIE,NaN
5441,tt11803618,tm898842,C/O Kaadhal,NaN,NaN,2021,MOVIE,NaN
5442,tt14585902,tm1059008,Lokillo,NaN,NaN,2021,MOVIE,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5444 entries, 0 to 5443
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   movie_id             5444 non-null   object 
 1   tmdb_id              5444 non-null   object 
 2   title                5444 non-null   object 
 3   hours_viewed         1802 non-null   float64
 4   global_availability  1802 non-null   object 
 5   release_year         5444 non-null   int64  
 6   type                 5444 non-null   object 
 7   age_certification    3068 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 340.4+ KB


Netflixlibcredits: the table contains actor,movie,show information

In [78]:
netflixlibcredits.info()
#only leaves rows of movies that already exist within the engagement (also netflix library) table
netflixlibcredits = netflixlibcredits[netflixlibcredits['tmdb_id'].isin(engagement['tmdb_id'])]
#remove occurences where the actor plays different characters in the same movie/show
netflixlibcredits = netflixlibcredits.drop_duplicates(subset=['person_id', 'tmdb_id','role'], keep='first')

display(netflixlibcredits)
netflixlibcredits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77783 entries, 0 to 77782
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77783 non-null  int64 
 1   tmdb_id    77783 non-null  object
 2   name       77783 non-null  object
 3   role       77783 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.4+ MB


,person_id,tmdb_id,name,role
0,3748,tm84618,Robert De Niro,ACTOR
1,14658,tm84618,Jodie Foster,ACTOR
2,7064,tm84618,Albert Brooks,ACTOR
3,3739,tm84618,Harvey Keitel,ACTOR
4,48933,tm84618,Cybill Shepherd,ACTOR
...,...,...,...,...
77778,736339,tm1059008,Adelaida Buscato,ACTOR
77779,399499,tm1059008,Luz Stella Luengas,ACTOR
77780,373198,tm1059008,Inés Prieto,ACTOR
77781,378132,tm1059008,Isabel Gaona,ACTOR


<class 'pandas.core.frame.DataFrame'>
Index: 74175 entries, 0 to 77782
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  74175 non-null  int64 
 1   tmdb_id    74175 non-null  object
 2   name       74175 non-null  object
 3   role       74175 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.8+ MB


fact_imdb_ratings: which is going to be our fact table, we added the forein keys and their respective data.

In [79]:
ratings = ratings.rename(columns={"averageRating": "average_rating"})
#duplicate the id column which will later serve as foreign keys
ratings.insert(1,"movie_id",ratings.imdb_id)
ratings.insert(2,"revenue_id",ratings.imdb_id)
ratings.insert(3,"genre_id",ratings.imdb_id)
ratings.insert(4,"region_id",ratings.imdb_id)
ratings.info()
#only keeping rows from imdb_ratings that are already existing in the netflix library
ratings = ratings[ratings['movie_id'].isin(engagement['movie_id'])]
#We're going to replace unapplicable foreign key rows with null values (such as revenue data for non movies)
missing_rows = ratings[~ratings['movie_id'].isin(movies_revenues['revenue_id'])]
ratings.loc[missing_rows.index, 'revenue_id'] = pd.NA

display(ratings)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1392739 entries, 0 to 1392738
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   imdb_id         1392739 non-null  object 
 1   movie_id        1392739 non-null  object 
 2   revenue_id      1392739 non-null  object 
 3   genre_id        1392739 non-null  object 
 4   region_id       1392739 non-null  object 
 5   average_rating  1392739 non-null  float64
dtypes: float64(1), object(5)
memory usage: 63.8+ MB


,imdb_id,movie_id,revenue_id,genre_id,region_id,average_rating
28165,tt0047673,tt0047673,<NA>,tt0047673,tt0047673,7.6
29951,tt0049761,tt0049761,<NA>,tt0049761,tt0049761,6.7
31353,tt0051390,tt0051390,<NA>,tt0051390,tt0051390,7.5
34481,tt0054953,tt0054953,tt0054953,tt0054953,tt0054953,7.5
36630,tt0057357,tt0057357,<NA>,tt0057357,tt0057357,7.5
...,...,...,...,...,...,...
1391975,tt9897038,tt9897038,NaN,tt9897038,tt9897038,6.2
1392054,tt9898858,tt9898858,NaN,tt9898858,tt9898858,5.2
1392088,tt9899340,tt9899340,NaN,tt9899340,tt9899340,7.7
1392382,tt9908860,tt9908860,NaN,tt9908860,tt9908860,7.2


<class 'pandas.core.frame.DataFrame'>
Index: 5242 entries, 28165 to 1392709
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb_id         5242 non-null   object 
 1   movie_id        5242 non-null   object 
 2   revenue_id      298 non-null    object 
 3   genre_id        5242 non-null   object 
 4   region_id       5242 non-null   object 
 5   average_rating  5242 non-null   float64
dtypes: float64(1), object(5)
memory usage: 415.7+ KB


region_language:

7. Final step: We will export our data files into CSV files in a separate folder

In [80]:
import os
if not os.path.exists("new_data"):
    os.makedirs("new_data")
    
engagement.to_csv('new_data/new_engagement.csv',index=False)
genres.to_csv('new_data/new_genres.csv',index=False)
movies_revenues.to_csv('new_data/new_revenues.csv',index=False)
ratings.to_csv('new_data/new_imdb_ratings.csv',index=False)
netflixlibcredits.to_csv('new_data/new_credits.csv',index=False)
regions.to_csv('new_data/new_regions.csv',index=False)

engagement.info()
genres.info()
movies_revenues.info()
ratings.info()
netflixlibcredits.info()
regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5444 entries, 0 to 5443
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   movie_id             5444 non-null   object 
 1   tmdb_id              5444 non-null   object 
 2   title                5444 non-null   object 
 3   hours_viewed         1802 non-null   float64
 4   global_availability  1802 non-null   object 
 5   release_year         5444 non-null   int64  
 6   type                 5444 non-null   object 
 7   age_certification    3068 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 340.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5444 entries, 0 to 5443
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genre_id  5444 non-null   object
 1   genres    5444 non-null   object
dtypes: object(2)
memory usage: 85.2+ KB
<class 'pandas.core.frame.DataF